In [ ]:
import sys
import numpy as np
import torch
from torch import Tensor, ones, stack, load
from torch.autograd import grad
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import figure
import pandas as pd
from torch.nn import Module
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from scipy import stats
from pathlib import Path
import wandb
import time
from tesladatano import TeslaDatasetNo

from tqdm import trange
from tqdm.autonotebook import tqdm

# Import PINNFramework etc.
# https://github.com/ComputationalRadiationPhysics/NeuralSolvers.git
# sys.path.append("...")# PINNFramework etc.
# sys.path.append("/home/hoffmnic/Code/NeuralSolvers")
sys.path.append("NeuralSolvers")  
import PINNFramework as pf

## Preprocessing


In [ ]:
# Use cuda if it is available, else use the cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# login wandb (optional)
wandb.login()

# Initialize WandB (optional)
wandb.init(name='NO_reg_v7', 
           project='Neural_Operator_project',
           notes='squared sum of gradients', 
           #tags=['...'],
           #entity='...'
           )


# WandB Configurations (optional)
wandb.config.normalize = 1000  
wandb.config.batch_size= 4096   
wandb.config.lr = 1e-4
wandb.config.alpha = 100000



# Create instance of the dataset
ds = TeslaDatasetNo(diff = "fwd_diff", device = device, data ='train', normalize = wandb.config.normalize, rel_time = True)
ds_test = TeslaDatasetNo(device = device, ID = -1, data = "test",normalize = wandb.config.normalize, rel_time = True)


wandb.config.input_size = 6
wandb.config.output_size = 1
wandb.config.hidden_size = 100
wandb.config.num_hidden = 4
wandb.config.lb = ds.lb
wandb.config.ub = ds.ub

# trainloader
train_loader = DataLoader(ds, batch_size=wandb.config.batch_size,shuffle=True)
validloader = DataLoader(ds_test, batch_size=wandb.config.batch_size,shuffle=True)

model = pf.models.MLP(input_size=wandb.config.input_size,
                      output_size=wandb.config.output_size, 
                      hidden_size=wandb.config.hidden_size, 
                      num_hidden=wandb.config.num_hidden, 
                      lb=wandb.config.lb, 
                      ub=wandb.config.ub,
                      #activation = torch.relu
                      )



# model = pf.models.MLP(input_size=6,
#                       output_size=1, 
#                       hidden_size=100, 
#                       num_hidden=4, 
#                       lb=ds.lb, 
#                       ub=ds.ub,
#                       #activation = torch.relu
#                       )

model.to(device)


# optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=wandb.config.lr)
criterion = torch.nn.MSELoss()

# Log the network weight histograms (optional)
wandb.watch(model)

In [ ]:
ds.df0.shape

In [ ]:
def derivative(x, u):

    grads = ones(u.shape, device=u.device) # move to the same device as prediction
    grad_u = grad(u, x, create_graph=True, grad_outputs=grads )[0]
   
    # calculate first order derivatives
    #u_t = grad_u[:, 4]
    #u_t = u_t.reshape(-1, 1)
    
    #u_t = torch.sum(grad_u, 1).reshape(-1, 1)
    
    return grad_u

## Function for saving a checkpoint during training

In [ ]:
def write_checkpoint(checkpoint_path, epoch, min_mlp_loss, optimizer):
    checkpoint = {}
    checkpoint["epoch"] = epoch
    checkpoint["minimum_pinn_loss"] = min_mlp_loss
    checkpoint["optimizer"] = optimizer.state_dict()
    checkpoint["mlp_model"] = model.state_dict()
    torch.save(checkpoint, checkpoint_path)

## Training


In [ ]:
epochs = 3000
min_mlp_loss = np.inf
min_valid_loss = np.inf

x_data_plot=[]
y_data_all_plot=[]
y_data_1_plot=[]
y_data_2_plot=[]

# Set fixed random number seed
torch.manual_seed(1234)

begin = time.time()
for epoch in range(epochs):
# with trange(epochs, unit="epochs") as pbar:
#     for epoch in pbar:
        # Set current and total loss value
        current_loss = 0.0
        total_loss = 0.0
        total_loss1 = 0.0
        total_loss2 = 0.0

        model.train()   # Optional when not using Model Specific layer
        for i, data in enumerate(train_loader,0):
            #print(i)
            x_batch, y_batch = data
            #print('data', x_batch.shape)
            if wandb.config.batch_size == 1:
                x_batch=torch.squeeze(x_batch)
                y_batch=torch.squeeze(y_batch)
              #delta_t=torch.squeeze(delta_t)

            # Ground-truth temperature
            true_temp = x_batch[:,4].detach().clone()

            optimizer.zero_grad()
            x_batch.requires_grad=True #new
            pred = model(x_batch.to(device))
            
            u_deriv = derivative(x_batch,pred) #new
            loss1 = criterion(pred,y_batch.to(device))
            
            #target2 = torch.zeros_like(u_t) #new
            #loss2 = criterion(target2.to(device),u_t.to(device))*wandb.config.alpha #new
            loss2 = torch.mean(u_deriv**2) * wandb.config.alpha #new new
            loss = loss1 + loss2 #new

            loss.backward()
            optimizer.step()

            # Print statistics
            current_loss += loss.item()
            total_loss += loss.item()
            total_loss1 += loss1.item()
            total_loss2 += loss2.item()

            if i % 50 == 49:
#                 print('Loss after mini-batch %5d: %.8f' %
#                      (i + 1, current_loss / 50))
                current_loss = 0.0

        train_loss = total_loss/(i+1)
        loss1 = total_loss1/(i+1) #new
        loss2 = total_loss2/(i+1) #new
        #print("Epoch ", epoch, "Total Loss ", Loss)
        #pbar.set_postfix(loss=Loss)
        x_data_plot.append(epoch)
        y_data_all_plot.append(train_loss)      
        y_data_1_plot.append(loss1)
        y_data_2_plot.append(loss2)
        
            ######## new
        valid_loss = 0.0
        model.eval()     # Optional when not using Model Specific layer
        for j, data in enumerate(validloader,0):
            x_batch, y_batch = data
            # Forward Pass
            target = model(x_batch.to(device))
            # Find the Loss
            loss = criterion(target,y_batch.to(device))
            # Calculate Loss
            valid_loss += loss.item()
    
        print(f'Epoch {epoch} \t Training Loss: {\
        train_loss:.5f} \t Loss 1: {loss1:.5f} \t Loss 2: {loss2:.5f} \t Validation Loss: {valid_loss / (j+1):.5f}')    


        # uncomment for saving the best model and checkpoints during training
        # save best model
        if min_valid_loss > valid_loss:
            print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
            min_valid_loss = valid_loss

            # Saving State Dict    
            model_name_path = Path('nomodel/best_model_{}_{}.pt'.format(wandb.run.id, wandb.run.name))
            torch.save(model.state_dict(), model_name_path)

        # writing checkpoint
        if (epoch + 1) % 200 == 0:
            checkpoint_path = Path('nomodel/checkpoint_{}_{}_{}.pt'.format(wandb.run.id, wandb.run.name, epoch))
            write_checkpoint(checkpoint_path, epoch, min_valid_loss, optimizer)

        # Log the loss and accuracy values at the end of each epoch
        wandb.log({
            "Epoch": epoch,
            "Total Loss": train_loss,
            "Loss1 (temperature)": loss1,
            "Loss2 (regulariser)": loss2,
            "Validation Loss": valid_loss
            })
end = time.time()
print("time:", end - begin)        

In [ ]:
train_loader.dataset.x.device

In [ ]:
# Make the plot of Total Loss vs epochs
dpi = 360
figure(figsize=(10, 8), dpi = dpi)

plt.plot(x_data_plot,y_data_all_plot)
plt.title('Total training loss vs epoch')
plt.xlabel('Epoch')
plt.ylabel('Total Loss')
plt.show()


# Make the plot of the supervised loss
figure(figsize=(10, 8), dpi = dpi)
plt.plot(x_data_plot,y_data_1_plot)
plt.title('Supervised loss vs epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss1')
plt.show()

# Make the plot of time stability loss
figure(figsize=(10, 8), dpi = dpi)
plt.plot(x_data_plot,y_data_2_plot)
plt.title('Regulariser loss vs epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss2')
plt.show()

## Postprocessing -- Load the model


In [ ]:
# Load the best model
PATH = 'nomodel/best_model_{}_{}.pt'.format(wandb.run.id, wandb.run.name)

## Neural Operator without relative time trained on central differences
#PATH = '/content/drive/MyDrive/NeuralSolvers-heat-eqn/examples/Research project/nomodel/norl/cdiff/best_model_icxc395f_NO_run.pt'

## Neural Operator with relative time trained on central differences
#PATH = "/content/drive/MyDrive/nomodel/nomodel/rel_time/ctrl/best_model_3ef5thtw_NO_run.pt" #rl time ctrl

## Neural Operator with relative time trained on forward differences
#PATH = "/content/drive/MyDrive/nomodel/nomodel/rel_time/fwd/best_model_3cutaabz_NO_run.pt"  

# PATH ='nomodel/best_model_2yi86wor_NO_reg_v4.pt'

#PATH = 'nomodel/best_model_745m0jcl_NO_reg_v3.pt'

#PATH = 'nomodel/best_model_2zbvr34o_NO_reg_v2.pt'

model.load_state_dict(torch.load(PATH))
model.eval()

## Performance of the model on training data


In [ ]:
# Make a prediction
normalize = wandb.config.normalize
pred = model(ds.x.float().to(device)) #GPU
pred = pred.detach().cpu().numpy()/normalize

# ground-truth
gt = ds.y.cpu().numpy()/normalize

# Some statistics on the model performance on all of dataset
mae = np.sum(np.abs(pred - gt).mean(axis=None))
print('MAE:', mae)

mse = ((gt - pred)**2).mean(axis=None)
print('MSE:', mse)

rel_error = np.linalg.norm(pred - gt) / np.linalg.norm(gt)
print('Relative error (%):', rel_error*100)

figure(figsize=(10, 8), dpi= 360)

plt.plot(pred, '--')
plt.plot(gt, '-')
plt.legend(['Prediction', 'Ground-truth'])
plt.title('Prediction accuracy on training data')
plt.xlabel('t-idx')
plt.ylabel('ΔTemp/Δt (°C/s)')
plt.show()


# figure(figsize=(10, 8), dpi= 360)
# #time
# t=ds.t
# plt.plot(t,pred, '--')
# plt.plot(t,gt, '-')
# plt.title('Prediction accuracy on training data')
# plt.legend(['Prediction', 'Ground-truth'])
# plt.xlabel('time (seconds)')
# plt.ylabel('ΔTemp/Δt (°C/s)')
# plt.show()

## Function for evaluating the performance on the test data


In [ ]:
def evaluate(idd,rel_time,diff):
  print('ID = ', idd)
  # import test data
  ds_test = TeslaDatasetNo(device = device, ID = idd, data = "test",rel_time = rl, diff = "central_diff")

  # Prediction accuracy of the Neural Operator
  print('Prediction accuracy of the Neural Operator (NO)')
  begin = time.time()
  pred_der = model(ds_test.x.to(device))
  pred_der = pred_der.detach().cpu().numpy()/normalize
  end = time.time()
  true_der = ds_test.y.cpu().numpy()
  print("time:", end - begin)
  
  # relative time
  t=ds_test.t

  #MAE
  mae = np.sum(np.abs(pred_der- true_der).mean(axis=None))
  print('MAE:', mae)

  #MSE
  mse = ((true_der - pred_der)**2).mean(axis=None)
  print('MSE:', mse)

  #Relative error
  rel_error = np.linalg.norm(pred_der - true_der) / np.linalg.norm(true_der)
  print('Relative error (%):', rel_error*100)
  plt.figure(figsize = (12, 8))
  plt.plot(t, pred_der, '-', label='Prediction')
  plt.plot(t, true_der, '--', label='Ground-truth')
  plt.title('Prediction accuracy of Neural Operator vs ground-truth for drive-ID = {}'.format(idd))
  plt.xlabel('t (seconds)')
  plt.ylabel('ΔTemp/Δt (°C/s)')
  plt.grid()
  plt.legend(loc='lower right')
  plt.show()
  
  print('########################################################')
  
  #3)Forward Euler method with fixed initial env. conditions but with updated 
  #Temperature (and rel time) from the prediction of the model at previous iteration
  #with generated temporally equidistant time steps

  print('Forwad Euler method with fixed initial env conditions')
  rel_t = ds_test.rel_t

  # ground-truth time
  t=ds_test.t
  max_t = t.max()
  t=t.cpu().numpy()

  # Ground-truth temperature
  true_temp = ds_test.x[:,4].cpu().numpy()

  # Predicted temperature using model prediction and forward euler method
  pred_temp = np.zeros((ds_test.x.shape[0]))
  pred_temp = true_temp.copy()

  # Fixed initial conditions for all environmental conditions
  input = ds_test.x[0].detach().clone()

  # temporally equdistant time steps
  tt = np.linspace(0,max_t,ds_test.x.shape[0])
  step_size=tt[2]-tt[1]

  #ODE
  begin = time.time()

  for i in range(0, ds_test.x.shape[0] - 1):
      input[4] = torch.tensor(pred_temp[i]).detach().clone()
      if rel_time == True:
        input[5] = torch.tensor(rel_t[i]).detach().clone()
      pred = model(input.to(device))
      pred = pred.detach().cpu().numpy()/normalize
      pred_temp[i + 1] = pred_temp[i] + pred*step_size
  end = time.time()

  print("time:", end - begin)

  #MAE
  mae = np.sum(np.abs(pred_temp- true_temp).mean(axis=None))
  print('MAE:', mae)

  #MSE
  mse = ((true_temp - pred_temp)**2).mean(axis=None)
  print('MSE:', mse)

  #Relative error
  rel_error = np.linalg.norm(pred_temp - true_temp) / np.linalg.norm(true_temp)
  print('Relative error (%):', rel_error*100)

  plt.figure(figsize = (12, 8))
  plt.plot(tt, pred_temp, '-', label='Prediction')
  plt.plot(t, true_temp, '--', label='Ground-truth')
  plt.title('Prediction vs ground-truth for drive-ID = {} (temporally equidistant step size)'.format(idd))
  plt.xlabel('t (seconds)')
  plt.ylabel('Temperature (°C)')
  plt.grid()
  plt.legend(loc='lower right')
  plt.show()

  #4)Forward Euler method with updated environmental conditions from the dataset at each iteration
  #But with updated temperature from the prediction of the model at previous iteration
  #with true step sizes
  print('Forwad Euler method with updated env conditions from the dataset at each iteration with true step sizes')
  
  # time
  t=ds_test.t
  t=t.numpy()
  
  # max time
  max_t = t.max()

  # Ground-truth temperature
  true_temp = ds_test.x[:,4].cpu().numpy()

  # Predicted temperature using model prediction and forward euler method
  pred_temp = np.zeros((ds_test.x.shape[0]))
  pred_temp[0] = true_temp[0].copy()

  begin = time.time()
  for i in range(0, ds_test.x.shape[0] - 1):
        input = ds_test.x[i].detach().clone()
        input[4] = torch.tensor(pred_temp[i]).detach().clone()
        pred = model(input.to(device))
        pred = pred.detach().cpu().numpy()/normalize
        pred_temp[i + 1] = pred_temp[i] + pred*(t[i+1]-t[i])
  end = time.time()


  print("time:", end - begin)
  #MAE 
  mae = np.sum(np.abs(pred_temp- true_temp).mean(axis=None))
  print('MAE:', mae)

  #MSE
  mse = ((true_temp - pred_temp)**2).mean(axis=None)
  print('MSE:', mse)

  # Relative error
  rel_error = np.linalg.norm(pred_temp - true_temp) / np.linalg.norm(true_temp)
  print('Relative error (%):', rel_error*100)

  plt.figure(figsize = (12, 8))
  plt.plot(t, pred_temp, '-', label='Prediction')
  plt.plot(t, true_temp, '--', label='Ground-truth')
  #plt.title('Approximate (with updated env conditions) and True Solution (true step size)'.format(idd))
  plt.title('Prediction (with updated env. conditions) vs ground-truth for drive-ID = {} (true step size)'.format(idd))
  plt.xlabel('t (seconds)')
  plt.ylabel('Temperature (°C)')
  plt.grid()
  plt.legend(loc='lower right')
  plt.show()


  # #5 time
  # plt.figure(figsize = (12, 8))
  # plt.plot(tt, '-', label='equidistant step size')
  # plt.plot(t, '--', label='true stepsize')
  # plt.title('Plot of equidistant test time vs true time')
  # plt.legend(loc='lower right')

  # #6
  # plt.figure(figsize = (12, 8))
  # plt.plot(np.diff(tt.reshape(-1)),'-', label='equidistant step size')
  # plt.plot(np.diff(t.reshape(-1)),  '--', label='true step-size')
  # plt.title('Plot of equidistant test time step size vs true time step size')
  # plt.legend(loc='upper left')


## Performance of the model on test data


In [ ]:
# Test values = [16,39,47,52,72,81,88]
rl = True
diff = "fwd_diff"

evaluate(idd=16,rel_time=rl,diff=diff)

In [ ]:
# Test values = [16,39,47,52,72,81,88]
evaluate(idd=39,rel_time=rl,diff=diff)

In [ ]:
# Test values = [16,39,47,52,72,81,88]
evaluate(idd=47,rel_time=rl,diff=diff)

In [ ]:
# Test values = [16,39,47,52,72,81,88]
evaluate(idd=52,rel_time=rl,diff=diff)

In [ ]:
# Test values = [16,39,47,52,72,81,88]
evaluate(idd=72,rel_time=rl,diff=diff)

In [ ]:
# Test values = [16,39,47,52,72,81,88]
evaluate(idd=81,rel_time=rl,diff=diff)

In [ ]:
# Test values = [16,39,47,52,72,81,88]
evaluate(idd=88,rel_time=rl,diff=diff)